In [4]:
import numpy as np
import scipy.io as sio
import datetime
import tqdm
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn as sns
sns.set(style="ticks", color_codes=True)
from itertools import combinations 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
thr = 1
dyR = 1 
[total_PED_sensors_winds_mult,total_ind] = calculate4DmatOfPED(Q_source,sensorArray,thr,dyR)

In [9]:
windfrequencies7x8RPHaderawithSC = pd.read_csv('/Users/iditbela/Documents/Shai_Kendler/Borg-1.9/meteorological_data/wind_frequencies_7x8_RP_Hadera_with_SC.csv')

In [30]:
windfrequencies7x8RPHaderawithSC.head(12)
np.shape(windfrequencies7x8RPHaderawithSC)[0]

,Unnamed: 0,s,d,sc,count,percent
0,0,0-1,337.5-22.5,A,254.0,0.000325
1,1,0-1,337.5-22.5,B,160.0,0.000205
2,2,0-1,337.5-22.5,C,164.0,0.000210
3,3,0-1,337.5-22.5,D,280.0,0.000358
4,4,0-1,337.5-22.5,E,182.0,0.000233
5,5,0-1,337.5-22.5,F,445.0,0.000570
6,6,0-1,22.5-67.5,A,237.0,0.000303
7,7,0-1,22.5-67.5,B,178.0,0.000228
8,8,0-1,22.5-67.5,C,278.0,0.000356
9,9,0-1,22.5-67.5,D,609.0,0.000780


336

In [45]:
    data = np.zeros((np.shape(windfrequencies7x8RPHaderawithSC)[0],4))
    windfrequencies = pd.DataFrame(data = data, columns = ['ws','wd','sc','perc'])
    ws_list = [1, 2, 4, 6, 8, 10, 20]
    ind = 0 
    for i in ws_list: # index it ends with 
        for j in np.arange(0,337.5,45):
            for k in range(1,7):
                windfrequencies.loc[ind,'ws']=i
                windfrequencies.loc[ind,'wd']=j
                windfrequencies.loc[ind,'sc']=k
                ind = ind + 1
windfrequencies['perc'] = windfrequencies7x8RPHaderawithSC['percent']
windfrequencies.head()

,ws,wd,sc,perc
0,1.0,0.0,1.0,0.000325
1,1.0,0.0,2.0,0.000205
2,1.0,0.0,3.0,0.000210
3,1.0,0.0,4.0,0.000358
4,1.0,0.0,5.0,0.000233


In [47]:
S,D,SC = np.meshgrid([0.5, 1.5, 3, 5, 7, 9, 12],[0, 45, 90, 135, 180, 225, 270, 315],[range(1,7)])

In [72]:
WF = windfrequencies['perc'].values.reshape(np.shape(D),order = 'F') # python's order is 'C' while Matlab's is 'F'

In [76]:
WF[1,0,0]

0.00020481389457460797

In [63]:
np.size(sensorArray,1)

6

In [17]:
# EDIT THIS FUNCTION LATER. SHOULD NOT BE A FUNCTION, BUT A SCRIPT
def calculate4DmatOfPED(Q_source,sensorArray,thr,dyR):

    ''' This script calculates in advance the 4D matrix of PED for each
    sensor placement and for each wind speed and direction. It calls the
    function calcSensorsPED which does it for a certain wind speed and
    direction (and average Q_source). Then it multiplies by wind frequencies. 
    The PED is just the .^2 of the norm between each 2 relevant states for each of the X possible sensors, 
    after taking into consideration the wind frequency.
    During the optimization I sum all
    relevant sensors out of X possible sensors and take the sqrt. Then I have 355x1 matrix
    from which I can calculate the mean of the min_PED or what ever function.'''
      
    thr = 1
    dyR = 1

    # wind frequencies speed: (<1, 1-2, 2-4, 4-6, 6-8, 8-10, >10) 
    # wind frequencies directions: 8 directions, 45 degrees each - north, east,
    # south, west and in between. 

    global total_readings # aggregate all readings to see range of values 
    global total_fields

    total_readings = []
    total_fields = []

    all_relevant_variations_size = 355 # relevant variations are only those of different number of active sources, not including moving from zero to x sources

    # load the original csv file and turn it to an array 
    # ALL (not seperating to day and night)
    data = np.zeros((np.shape(windfrequencies7x8RPHaderawithSC)[0],4))
    windfrequencies = pd.DataFrame(data = data, columns = ['ws','wd','sc','perc'])
    ws_list = [0.5, 1.5, 3, 5, 7, 9, 12]
    ind = 0 
    for i in ws_list: # ws I apply
        for j in np.arange(0,337.5,45): # wd I apply
            for k in range(1,7): # stability class
                windfrequencies.loc[ind,'ws']=i
                windfrequencies.loc[ind,'wd']=j
                windfrequencies.loc[ind,'sc']=k
                ind = ind + 1
                
    windfrequencies['perc'] = windfrequencies7x8RPHaderawithSC['percent']

    
    # NOT SURE ALL HERE IS NECESSARY AND NP.SIZE HAS ZERO INDEX AS WELL !
    
#     # create a matrix of wind speeds and directions 
#     S,D,SC = np.meshgrid([0.5, 1.5, 3, 5, 7, 9, 12],[0, 45, 90, 135, 180, 225, 270, 315],[range(1,7)])
#     # reshape windfrequencies to matrix 
#     WF = windfrequencies['perc'].values.reshape(np.shape(D))



    NumOfSens = np.size(sensorArray,1)
    total_PED_sensors_winds = zeros(size(WF,1),size(WF,2),size(WF,3),all_relevant_variations_size,NumOfSens);

    for i=1:size(S,2) # WS - 7
        for j=1:size(S,1) # WD - 8
            for k=1:size(S,3)
                WS = S(1,i,1)
                WD = D(j,1,1)
                ASC = SC(1,1,k)
                wf = WF(j,i,k);
                [total_PED_sensors,total_ind]=calcSensorsPED(Q_source,sensorArray,WD,WS,ASC,wf,thr,dyR);
                total_PED_sensors_winds(j,i,k,:,:)=total_PED_sensors;


    # NOTE - I think in this approach I give the weight to the already
    # calculated PED value. In the second approach, I generate average
    # concentration maps and then calculate the PED value. its the same if I
    # multiply in the following by WF^2
    # multiply by wind frequencies (deprecated, because it was not done in the right place)
    # for n=1:NumOfSens
    #   for m=1:all_relevant_variations_size
    #        total_PED_sensors_winds(:,:,m,n) = total_PED_sensors_winds(:,:,m,n).*WF; 
    #    end
    # end

    total_PED_sensors_winds_mult = squeeze(sum(sum(sum(total_PED_sensors_winds,1),2),3));

    
    
    return total_PED_sensors_winds_mult, total_ind

SyntaxError: invalid syntax (<ipython-input-17-cad6e8099711>, line 34)

In [225]:
# IN CASE I WILL WANT TO TRY NOT 0/1 TRAINING

# from itertools import combinations 

# comb = product([0.0, 0.5, 1.0, 1.5], repeat=5)

# cnt = 1
# # Print the obtained combinations 
# for i in list(comb): 
#     i
#     cnt = cnt + 1 
# cnt


In [243]:
total_active = np.sum(total_s,axis=1)
total_active

array([0., 1., 1., 2., 1., 2., 2., 3., 1., 2., 2., 3., 2., 3., 3., 4., 1.,
       2., 2., 3., 2., 3., 3., 4., 2., 3., 3., 4., 3., 4., 4., 5.])

In [247]:
np.shape(total_active)

(32,)

In [272]:
from itertools import combinations 

comb = list(combinations(range(len(total_active)), 2))
comb_diff_number = []
for i,j in comb:
    if total_active[i] != total_active[j]:
        comb_diff_number.append((i,j,total_active[i],total_active[j]))

# comb_diff_number
for i,j,m,n in comb_diff_number:
    PEDs.append(np.sqrt(np.sum((chosen_readings[:,i]-chosen_readings[:,j])**2)))


# cnt = 1
# # Print the obtained combinations 
# for i in list(comb): 
#     i
#     cnt = cnt + 1 
# cnt





0 1 0.0 1.0
0 2 0.0 1.0
0 3 0.0 2.0
0 4 0.0 1.0
0 5 0.0 2.0
0 6 0.0 2.0
0 7 0.0 3.0
0 8 0.0 1.0
0 9 0.0 2.0
0 10 0.0 2.0
0 11 0.0 3.0
0 12 0.0 2.0
0 13 0.0 3.0
0 14 0.0 3.0
0 15 0.0 4.0
0 16 0.0 1.0
0 17 0.0 2.0
0 18 0.0 2.0
0 19 0.0 3.0
0 20 0.0 2.0
0 21 0.0 3.0
0 22 0.0 3.0
0 23 0.0 4.0
0 24 0.0 2.0
0 25 0.0 3.0
0 26 0.0 3.0
0 27 0.0 4.0
0 28 0.0 3.0
0 29 0.0 4.0
0 30 0.0 4.0
0 31 0.0 5.0
1 3 1.0 2.0
1 5 1.0 2.0
1 6 1.0 2.0
1 7 1.0 3.0
1 9 1.0 2.0
1 10 1.0 2.0
1 11 1.0 3.0
1 12 1.0 2.0
1 13 1.0 3.0
1 14 1.0 3.0
1 15 1.0 4.0
1 17 1.0 2.0
1 18 1.0 2.0
1 19 1.0 3.0
1 20 1.0 2.0
1 21 1.0 3.0
1 22 1.0 3.0
1 23 1.0 4.0
1 24 1.0 2.0
1 25 1.0 3.0
1 26 1.0 3.0
1 27 1.0 4.0
1 28 1.0 3.0
1 29 1.0 4.0
1 30 1.0 4.0
1 31 1.0 5.0
2 3 1.0 2.0
2 5 1.0 2.0
2 6 1.0 2.0
2 7 1.0 3.0
2 9 1.0 2.0
2 10 1.0 2.0
2 11 1.0 3.0
2 12 1.0 2.0
2 13 1.0 3.0
2 14 1.0 3.0
2 15 1.0 4.0
2 17 1.0 2.0
2 18 1.0 2.0
2 19 1.0 3.0
2 20 1.0 2.0
2 21 1.0 3.0
2 22 1.0 3.0
2 23 1.0 4.0
2 24 1.0 2.0
2 25 1.0 3.0
2 26 1.0 3.0
2 27 

In [261]:
a = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20],[21,22,23,24]]
tuples_list = [(1,0), (2,3), (3,2), (2,0)]
for x,y in tuples_list:
    print(a[x][y])

5
12
15
9


In [185]:
# sensorArray = np.zeros((10,3))
# sensorArray[:,1]=np.arange(0,500,50)
# sensorArray

# Q_source = [0.47, 0.51, 0.38, 0.9, 0.19]
numOfSensors = np.size(sensorArray,0)
numOfSources = np.size(Q_source)
# Run all 32 options of working/not working stacks (for 5 sources) and calculate readings
# ((including the change in readings from zeros))
totalField = np.zeros((numOfSensors,2**numOfSources))
total_active = np.zeros((2**numOfSources,))

No_of_Bits = np.size(Q_source)
a = []
for bits in np.arange(0,2**No_of_Bits):
    a.append(np.binary_repr(bits,width=No_of_Bits))
total_s = [] 
for row in a:
    s = []
    for binary in row:
        s.append(float(binary))
    total_s.append(s)

for scenario, active in enumerate(total_s):
    sensorArray,_,_ = calculateDisp(Q_source*active, sensorArray, WD, WS, ASC); 
    total_active[scenario] = np.sum(active);
    totalField[:,scenario] = sensorArray[:,3] 
    

    

array([[  0.,   0.,   0.],
       [  0.,  50.,   0.],
       [  0., 100.,   0.],
       [  0., 150.,   0.],
       [  0., 200.,   0.],
       [  0., 250.,   0.],
       [  0., 300.,   0.],
       [  0., 350.,   0.],
       [  0., 400.,   0.],
       [  0., 450.,   0.]])

[0.47, 0.51, 0.38, 0.9, 0.19]

5

(10, 32)

In [234]:
No_of_Bits = 5
a = []
for bits in np.arange(0,2**No_of_Bits):
    a.append(np.binary_repr(bits,width=No_of_Bits))
total_s = [] 
for row in a:
    s = []
    for binary in row:
        s.append(float(binary))
    total_s.append(s)

In [241]:
np.sum(total_s,axis=1)

array([0., 1., 1., 2., 1., 2., 2., 3., 1., 2., 2., 3., 2., 3., 3., 4., 1.,
       2., 2., 3., 2., 3., 3., 4., 2., 3., 3., 4., 3., 4., 4., 5.])

In [280]:
# configurations to load
conf = { 
    # general
    'GRID_SIZE': 1000,
    'GRID_SCALE': 1,
    'EFFECTIVE_HEIGHT': 10,
    'NOISE_RANGE': 1e-16,

    # meteorological properties
    'WIND_SPEED': [],
    'WIND_DIRECTION': [],
    'STABILITY_PARAMETER_a': [],
    'STABILITY_PARAMETER_c': [],
    'STABILITY_PARAMETER_d': [],
    'STABILITY_PARAMETER_f': [],

    # source locations (should be configured according to number of sources)
    'SOURCE_1': [],
    'SOURCE_2': [],
    'SOURCE_3': [],
    'SOURCE_4': [],
    'SOURCE_5': [],

    # leak rates[kg/sec]
    'Q_SOURCE_1': [],
    'Q_SOURCE_2': [],
    'Q_SOURCE_3': [],
    'Q_SOURCE_4': [],
    'Q_SOURCE_5': []
}

In [278]:
class Config(object):
    def __init__(self):
        self._config = conf # set it to conf

    def get_property(self, property_name):
        if property_name not in self._config.keys(): # we don't want KeyError
            return None  # just return None if not found
        return self._config[property_name]

    def set_property(self, property_name, value):
        self._config[property_name] = value
    

In [ ]:
class API:
    
    # properties
    configFile = Config()
    
    # methods
    def distance(*arg, x1_cor, y1_cor, x2_cor, y2_cor):
        d = np.sqrt((x1_cor-x2_cor)**2 + (y1_cor-y2_cor)**2)   
        return d
    
    def getCurveByWindDirection(*arg, windDirection):
        '''the function returns the slope of the vector from wind direction [degrees].
        alpha is the angle between the vector of the wind and the positive direction of X axis'''
        omega = windDirection
        alpha = 270 - omega 
        # convert to radians
        alpha = np.deg2rad(alpha)
        m = np.tan(alpha)      
        return m
    
        # return the equation of a line by the slope and known point
        # the line is represented by m(the slope) and n (cut-off point with Y axis)
        # the line equation is : y-y1 = m*(x-x1) =>
        # y = m*(x-x1) + y1 =>
        # n = -x1*m + y1
    def getLineEquation(*arg, x1_cor, y1_cor, m):
        '''the function returns the equation of a line by the slope=m and known point (x1,y1).
        n is the intercept with Y axis. the line equation is: y-y1 = m*(x-x1) => y = m*(x-x1)+y1, 
        and when x=0 we get n:'''
        n = y1_cor-m*x1_cor           
        return m, n

#       get distance between line and point. the line represented by m
#       and n, the point is x and y
#       the equation is: d = (abs (-mx0 + y0 - n))/(sqrt(m^2 +1))

    def getDistanceFromPointToLine(*arg, m, n, x0, y0):
        '''get distance between a line and a point, the line is represented by m (slope) and n (equation),
        the point is x and y. the equation is d'''
        d = (np.abs(-m*x0+y0-n))/(np.sqrt(m**2+1)) # 1 is the coefficient of y in the line equation
        return d
    
    def calculateSensorCon(obj, x_sen, y_sen, x_source, y_source, sourceCon):
        '''calculate the sensor concentration level based on given emission from source'''
        Q = sourceCon
        U = obj.configFile.get_property('WIND_SPEED')
        wind_direction = obj.configFile.get_property('WIND_DIRECTION')
        # effective stack height, aribtrrary 
        He = obj.configFile.get_property('EFFECTIVE_HEIGHT')
        grid_scale = obj.configFile.get_property('GRID_SCALE')
        
        # first: calculate the distance between the source and sensor
        # deal with all winds directions   
        if(np.mod(180-wind_direction-np.rad2deg(np.arctan2(y_sen - y_source, x_sen - x_source)), 360) < 180):         
            return 0
                
        # X distance: In meters
        x_distance_meters  = obj.distance(x_sen, y_sen, x_source, y_source)*grid_scale
        x_distance_kilometers = (x_distance_meters/1000)

        # the slope of the wind direction refere to X axis
        mWD = obj.getCurveByWindDirection(wind_direction)

        # get the line equation of the wind, based on two points (the source) and the slope
        [mWD,nWD] =  obj.getLineEquation(x_source, y_source, mWD)

        #calculate the distance between the sensor and the line
        #equation of the wind direction
        y_distance = (obj.getDistanceFromPointToLine(mWD, nWD, x_sen, y_sen))

        # in meters
        y_distance =  y_distance*grid_scale
                                
        # calculate sigma y and sigma z                
        a = obj.configFile.get_property('STABILITY_PARAMETER_a')
        c = obj.configFile.get_property('STABILITY_PARAMETER_c')
        d = obj.configFile.get_property('STABILITY_PARAMETER_d')
        f = obj.configFile.get_property('STABILITY_PARAMETER_f')
                  
        sigma_y = a*(x_distance_kilometers**0.894)              
        sigma_z = c*(x_distance_kilometers**d) + f                
                
        C = (Q/(U*sigma_y*sigma_z*np.pi))*(np.exp(-(y_distance**2)/(2*(sigma_y**2)))*np.exp(-(He**2)/(2*(sigma_z**2))))
        return C
    
    def addGausienNoise(obj, oldSignal):
        # noise level
        noiseRange = obj.configFile.get_property('NOISE_RANGE')
        noiseLevelA = -(noiseRange)
        noiseLevelB = noiseRange
        noise  = noiseLevelA + (noiseLevelB - noiseLevelA)*rand

        newSignal = oldSignal + noise*oldSignal

        return newSignal
         

In [ ]:
def calculateDisp(Q_source,sensorArray, WD, WS, SC): # in matlab called idit_CD1
    
    svAPI = API()
    svAPI.configFile.set_property('WIND_DIRECTION', WD)
    svAPI.configFile.set_property('WIND_SPEED', WS)
    
    # stability parameters according to Martin 1976
    if SC == 1:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',213)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',440.8)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',1.941)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',9.27)

    if SC == 2:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',156)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',106.6)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',1.149)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',3.3)

    if SC == 3:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',104)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',61)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',0.911)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',0)

    if SC == 4:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',68)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',33.2)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',0.725)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',-1.7)

    if SC == 5:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',50.5)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',22.8)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',0.678)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',-1.3)

    if SC == 6:
        svAPI.configFile.set_property('STABILITY_PARAMETER_a',34)
        svAPI.configFile.set_property('STABILITY_PARAMETER_c',14.35)
        svAPI.configFile.set_property('STABILITY_PARAMETER_d',0.740)
        svAPI.configFile.set_property('STABILITY_PARAMETER_f',-0.035) 
    
    configFile = Config()
    sizeOfStudyArea = configFile.get_property('GRID_SIZE')
    
    # Source locations (!!! maybe try to enable flexibility in number of sources and locations!!!)
    x_source1 = configFile.get_property('SOURCE_1')[0]
    y_source1 = configFile.get_property('SOURCE_1')[1]

    x_source2 = configFile.get_property('SOURCE_2')[0]
    y_source2 = configFile.get_property('SOURCE_2')[1]

    x_source3 = configFile.get_property('SOURCE_3')[0]
    y_source3 = configFile.get_property('SOURCE_3')[1]

    x_source4 = configFile.get_property('SOURCE_4')[0]
    y_source4 = configFile.get_property('SOURCE_4')[1]

    x_source5 = configFile.get_property('SOURCE_5')[0]
    y_source5 = configFile.get_property('SOURCE_5')[1]
    
    x_source = np.array[x_source1,x_source2,x_source3,x_source4,x_source5]
    y_source = np.array[y_source1,y_source2,y_source3,y_source4,y_source5]
        
    sourceArray = np.zeros((np.size(Q_source),3))
    for i in range(np.size(Q_source)):
        sourceArray[i,1] = x_source[i] # should change the above code !!!!
        sourceArray[i,2] = y_source[i]
        sourceArray[i,3] = Q_source[i]
    
    for i in range(np.size(sensorArray,0)):
        totalAmbientDataOfSensor = 0
        for j in range(np.size(sourceArray,0)):
            ambientDataFromOneSource = svAPI.calculateSensorCon(sensorArray[i,1], sensorArray[i,2], sourceArray[j,1], sourceArray[j,2], sourceArray[j,3])
            ambientDataFromOneSource = svAPI.addGausienNoise(ambientDataFromOneSource)
            totalAmbientDataOfSensor  = totalAmbientDataOfSensor + ambientDataFromOneSource
        sensorArray[i,3] = totalAmbientDataOfSensor
  
    return sensorArray,sourceArray,sizeOfStudyArea
    
    

In [ ]:
def calcSensorsReadings(Q_source,sensorArray,WD,WS,ASC):
    '''The function calculates the readings in places specified in sensorArray, 
    according to Q_source, WD, WS and ASC, FOR ALL POSSIBLE COMBINATIONS OF ACTIVE/INACTIVE Q.
    totalField is nXm matrix with n=number of sensors and m=all scenarios (2^(number of sources))
    total_s is all possible scenarios (0 = source inactive, 1=source active)''' 

    numOfSensors = np.size(sensorArray,0)
    numOfSources = np.size(Q_source)
    # Run all 32 options of working/not working stacks (for 5 sources) and calculate readings
    # ((including the change in readings from zeros))
    totalField = np.zeros((numOfSensors,2**numOfSources))

    No_of_Bits = np.size(Q_source)
    a = []
    for bits in np.arange(0,2**No_of_Bits):
        a.append(np.binary_repr(bits,width=No_of_Bits))
    total_s = [] 
    for row in a:
        s = []
        for binary in row:
            s.append(float(binary))
        total_s.append(s)

    for scenario, active in enumerate(total_s):
        sensorArray,_,_ = calculateDisp(Q_source*active, sensorArray, WD, WS, ASC)
        total_s[scenario] = np.sum(active)
        totalField[:,scenario] = sensorArray[:,2]
#     total_cr = totalField
    return totalField, total_s
    
def calcSensorsPED(totalField,total_s,sensorIdx,thr,dyR):
    '''WF???not sure what to do with it..., The function calculates all PEDs only for situations of different number of active sources, 
    given an array of sensor locations (location index(row) of sensorArray) and an array of sensors sensitivity 
    (thr) and dynamic range (accordingly). The function should be used during the optimization process.
    PEDs is nX1 matrix where n=number of situations with different number of active sources.
    Scenario pairs specifies each pair of scenarios corresponding to the PED row in PEDs and the 
    number of active/inactive sources in those scenarios (3,4) and number of scenario combination(1,2)''' 
    
    # choose the relevant rows(=sensors) according to sensorIdx
    chosen_readings = totalField[sensorIdx,:]
    numOfSensors = np.size(sensorIdx)
    numOfSources = np.size(total_s,1)
    total_active = np.sum(total_s,axis=1)
    
    # (comment these lines if thr and dyR not applied)
    # for each row (sensor), apply thr and dyR
    for i in range(numOfSensors):
        chosen_readings[i,chosen_readings[i,:]<thr[i]] = 0
        chosen_readings[i,chosen_readings[i,:]>thr[i]*dyR[i]] = thr[i]*dyR[i]

    # calculate PEDs for the chosen sensors
    PEDs = []
    comb = list(combinations(range(len(total_active)), 2))
    scenario_pairs = []
    for i,j in comb:
        if total_active[i] != total_active[j]:
            scenario_pairs.append((i,j,total_active[i],total_active[j]))
    for i,j,m,n in scenario_pairs:
        PEDs.append(np.sqrt(np.sum((chosen_readings[:,i]-chosen_readings[:,j])**2)))   
           
    return PEDs, scenario_pairs


In [630]:
def initializeSimulation(numOfSources, emissionRates, distanceBetweenSensors, distanceFromSource):   
    '''Initialize Q_source and sensorArray. sensorArray is given Nan in places to exclude
    numOfSources = number of sources defined in the configuration file, 
    emissionRates = an array of emission rates,
    distanceBetweenSensors = how many meters appart to place sensors, 
    distanceFromSource = radious [meters] around the source where I can't place my sensors at'''
    
    configFile = Config()
    boundery = configFile.get_property('GRID_SIZE')
    
    # get source locations 
    sourceLocations = np.zeros((numOfSources,2)) #numpy array
    for i in range(1,numOfSources+1): 
        sourceLocations[i-1,:] = configFile.get_property('SOURCE_'+ str(i))
    
    # set source emission rates
    Q_source = np.zeros((numOfSources))
    for i in range(1,numOfSources+1): 
        Q_source[i-1] = configFile.set_property('Q_SOURCE_'+ str(i),emissionRates[i-1])
    
    # place potential sensors all over the area, distanceBetweenSensors meter appart
    [X,Y] = np.meshgrid(np.arange(0,boundery+distanceBetweenSensors,distanceBetweenSensors), np.arange(0,boundery+distanceBetweenSensors,distanceBetweenSensors))
    
    # exclude sensors near sources (distanceFromSource to each direction)
    [exclude_x, exclude_y] = np.meshgrid(np.arange(-distanceFromSource,2*distanceFromSource,distanceFromSource),np.arange(-distanceFromSource,2*distanceFromSource,distanceFromSource))  
    exclude = np.zeros((np.size(exclude_x),2))
    exclude[:,0]=np.reshape(exclude_x,(np.size(exclude_x,0)*np.size(exclude_x,1),),order = 'F')
    exclude[:,1]=np.reshape(exclude_y,(np.size(exclude_y,0)*np.size(exclude_y,1),),order = 'F')

    sensors_to_exclude = [] #list
    for i in range(numOfSources):
        sensors_to_exclude.append(np.matlib.repmat(sourceLocations[i,:],np.size(exclude,0),1)+exclude)
    # convert to numpy array
    sensors_to_exclude = np.reshape(sensors_to_exclude,(np.size(exclude,0)*numOfSources,2))
    
    # all original sensors
    NumOfSensors = np.size(X)
    sensorArray = np.zeros((NumOfSensors,3))
    sensorArray[:,0]=np.reshape(X,(np.size(X,0)*np.size(X,1),),order = 'F')
    sensorArray[:,1]=np.reshape(Y,(np.size(Y,0)*np.size(Y,1),),order = 'F')
#     original_sensor_array = sensorArray


    # these are the sensors to exclude
    a = [(row == sensorArray[:,0:2]) for row in sensors_to_exclude]
    ind_to_exclude = []
    for i in range(np.shape(a)[0]):
        temp = a[:][i][:]
        for idx, j in enumerate(temp):
            if np.sum(j) == 2:
                ind_to_exclude.append(idx)
    
    sensorArray[ind_to_exclude,2] = np.nan
    
    return Q_source, sensorArray, sensors_to_exclude
        
    

In [602]:
# all original sensors
NumOfSensors = np.size(X)
sensorArray = np.zeros((NumOfSensors,3))
sensorArray[:,0]=np.reshape(X,(np.size(X,0)*np.size(X,1),),order = 'F')
sensorArray[:,1]=np.reshape(Y,(np.size(Y,0)*np.size(Y,1),),order = 'F')

sensorArray

array([[   0.,    0.,    0.],
       [   0.,   50.,    0.],
       [   0.,  100.,    0.],
       ...,
       [1000.,  900.,    0.],
       [1000.,  950.,    0.],
       [1000., 1000.,    0.]])

In [603]:
sensors_to_exclude = [] #list
for i in range(numOfSources):
    sensors_to_exclude.append(np.matlib.repmat(sourceLocations[i,:],np.size(exclude,0),1)+exclude)
# convert to numpy array
sensors_to_exclude = np.reshape(sensors_to_exclude,(np.size(exclude,0)*numOfSources,2))

# sensors_to_exclude

In [596]:
sourceLocations = np.array([[200,300],[300,700],[650,400],[450,200],[200,500]])

In [634]:
sourceLocations[0,0]

200

In [614]:
# a = [(row == sensors_to_exclude).any() for row in sensorArray[:,0:2]]
a = [(row == sensorArray[:,0:2]) for row in sensors_to_exclude]

In [615]:
np.shape(a)[0]

45

In [618]:
ind_to_exclude = []
for i in range(np.shape(a)[0]):
    temp = a[:][i][:]
    for idx, j in enumerate(temp):
        if np.sum(j) == 2:
            ind_to_exclude.append(idx)
                
#         print(np.where(np.any(temp[j],axis=0)))
#     print(np.any(temp,axis=0),i)

In [627]:
sensorArray[ind_to_exclude,2] 


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

In [454]:
sensors_to_exclude = np.reshape(sensors_to_exclude,(np.size(exclude,0)*numOfSources,2))

In [463]:
np.shape(np.unique(sensors_to_exclude,axis = 1))

(45, 2)

In [476]:
np.isin(sensorArray[68,0:2],sensors_to_exclude)

array([ True,  True])

In [608]:
idx = []
for i, row in enumerate(sensorArray[:,0:2]):   
    for sens_loc in sensors_to_exclude:        
        if np.isin(row,sens_loc).all():
            idx.append(i)
#             print(np.isin(row,sens_loc))
  

In [611]:
np.shape(np.unique(idx))

(85,)

In [436]:
sensors_to_exclude = [] #list
for i in range(numOfSources):
    sensors_to_exclude.append(np.matlib.repmat(sourceLocations[i,:],np.size(exclude,0),1)+exclude)

a = np.reshape(sensors_to_exclude,(9*5,2))

In [485]:
# sensors_to_exclude

In [486]:
# a

In [409]:
b = np.matlib.repmat(sourceLocations[i,:],np.size(exclude,0),1)+exclude
np.shape(b)

(9, 2)

In [415]:
sensors_to_exclude = np.array
np.shape(sensors_to_exclude)

(0,)

In [402]:
sensors_to_exclude = np.array([,])
for i in range(numOfSources):
    sensors_to_exclude = np.concatenate([sensors_to_exclude,np.matlib.repmat(sourceLocations[i,:],np.size(exclude,0),1)+exclude]) 




ValueError: all the input arrays must have same number of dimensions

In [396]:
np.shape(sensors_to_exclude)

(5, 9, 2)

In [449]:
np.unique(idx)

array([ 66,  68,  69,  70,  88,  89,  90,  91, 108, 109, 110, 111, 112,
       129, 130, 131, 132, 150, 151, 152, 154])

In [450]:
np.shape(np.unique(idx))

(21,)

In [446]:
all([True, True])

True

In [ ]:
[(v[i] in m[i]) for i in range(len(v))]

In [351]:
sensors_to_exclude

[array([[150., 250.],
        [150., 300.],
        [150., 350.],
        [200., 250.],
        [200., 300.],
        [200., 350.],
        [250., 250.],
        [250., 300.],
        [250., 350.]]), array([[150., 250.],
        [150., 300.],
        [150., 350.],
        [200., 250.],
        [200., 300.],
        [200., 350.],
        [250., 250.],
        [250., 300.],
        [250., 350.]]), array([[150., 250.],
        [150., 300.],
        [150., 350.],
        [200., 250.],
        [200., 300.],
        [200., 350.],
        [250., 250.],
        [250., 300.],
        [250., 350.]]), array([[150., 250.],
        [150., 300.],
        [150., 350.],
        [200., 250.],
        [200., 300.],
        [200., 350.],
        [250., 250.],
        [250., 300.],
        [250., 350.]]), array([[150., 250.],
        [150., 300.],
        [150., 350.],
        [200., 250.],
        [200., 300.],
        [200., 350.],
        [250., 250.],
        [250., 300.],
        [250., 350.]])]

In [ ]:
clc; close all; clear all;

NumOfVars = 5; % number of sources 
configFile = Configuration;
boundery = configFile.GRID_SIZE;

% Source emission rates 
Q_source(1:NumOfVars)=[configFile.Q_SOURCE_1 configFile.Q_SOURCE_2 configFile.Q_SOURCE_3 configFile.Q_SOURCE_4 configFile.Q_SOURCE_5];

% Source locations
sourceLocations = zeros(NumOfVars,2);
sourceLocations(1,:) = configFile.SOURCE_1; %first column X location, second column Y location
sourceLocations(2,:) = configFile.SOURCE_2;
sourceLocations(3,:) = configFile.SOURCE_3;
sourceLocations(4,:) = configFile.SOURCE_4;
sourceLocations(5,:) = configFile.SOURCE_5;

% place sensors all over the area
[X,Y] = meshgrid(0:50:boundery,0:50:boundery);

% Exclude sensors near sources (50 meters to each direction)
[exclude_x, exclude_y] = meshgrid(-50:50:50,-50:50:50);
exclude = zeros(numel(exclude_x),2);
exclude(:,1)=reshape(exclude_x,[size(exclude_x,1)*size(exclude_x,2),1]);
exclude(:,2)=reshape(exclude_y,[size(exclude_y,1)*size(exclude_y,2),1]);

sensors_to_exclude = [repmat(sourceLocations(1,:),[size(exclude,1),1])+exclude...
    ;repmat(sourceLocations(2,:),[size(exclude,1),1])+exclude...
    ;repmat(sourceLocations(3,:),[size(exclude,1),1])+exclude...
    ;repmat(sourceLocations(4,:),[size(exclude,1),1])+exclude...
    ;repmat(sourceLocations(5,:),[size(exclude,1),1])+exclude];

% all original sensors
NumOfSens = numel(X);
sensorArray = zeros(NumOfSens,3);
sensorArray(:,1)=reshape(X,[size(X,1)*size(X,2),1]);
sensorArray(:,2)=reshape(Y,[size(Y,1)*size(Y,2),1]);
original_sensor_array = sensorArray;

% these are the sensors to exclude
[a,b] = ismember(sensorArray(:,1:2),sensors_to_exclude,'rows');
sensorArray(a,:) = nan;


In [312]:
boundery = 1000
distanceBetweenSensors = 50
[X,Y] = np.meshgrid(np.arange(0,boundery+distanceBetweenSensors,distanceBetweenSensors), np.arange(0,boundery+distanceBetweenSensors,distanceBetweenSensors))

In [311]:
distanceFromSource = 50
[exclude_x, exclude_y] = np.meshgrid(np.arange(-distanceFromSource,2*distanceFromSource,distanceFromSource),np.arange(-distanceFromSource,2*distanceFromSource,distanceFromSource))

In [318]:
np.shape(exclude_x)

(3, 3)

In [ ]:
# constract a training set that is based on "known-hours" of emission (factors, from my excel)
# over these hours use wind data 
# have several types of sensors - different price, different "value" (dynamic range and detection thr) 
# evaluate based on distance from budget and PED (or other parameter?)

In [320]:
exclude = np.zeros((np.size(exclude_x),2))
exclude[:,0]=np.reshape(exclude_x,(np.size(exclude_x,0)*np.size(exclude_x,1),),order = 'F')
exclude[:,1]=np.reshape(exclude_y,(np.size(exclude_y,0)*np.size(exclude_y,1),),order = 'F')

In [342]:
np.size(X)

441

In [341]:
np.matlib.repmat(sourceLocations[0,:],np.size(exclude,0),1)+exclude

array([[150., 250.],
       [150., 300.],
       [150., 350.],
       [200., 250.],
       [200., 300.],
       [200., 350.],
       [250., 250.],
       [250., 300.],
       [250., 350.]])

In [336]:
sourceLocations = np.array([[200,300],[200,300],[200,300],[200,300],[200,300]])

In [327]:
np.shape(sourceLocations)

(5, 2)

In [331]:
np.size(exclude,0)

9

In [339]:
sourceLocations[0,:]

array([200, 300])

In [346]:
NumOfSensors = np.size(X)
sensorArray = np.zeros((NumOfSensors,3))
sensorArray[:,0]=np.reshape(X,(np.size(X,0)*np.size(X,1),),order = 'F')
sensorArray[:,1]=np.reshape(Y,(np.size(Y,0)*np.size(Y,1),),order = 'F')

In [347]:
sensorArray

array([[   0.,    0.,    0.],
       [   0.,   50.,    0.],
       [   0.,  100.,    0.],
       ...,
       [1000.,  900.,    0.],
       [1000.,  950.,    0.],
       [1000., 1000.,    0.]])